# Подготовка данных для построения модели

#### Начнем с простого, но важного шага. Отделите небольшую выборку от существующих данных. Назовем её hold-out dataset. Эта выборка нужна для контроля качества решения: она не должна использоваться вплоть до контроля качества решения. Наличие такой выборки поможет убедиться, что в процессе моделирования не было допущено ошибок, не произошло переобучение. В качестве ответа загрузите полученный файл (или файлы, если вы работаете а данными и метками как с 2мя файлами)

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
X = pd.read_csv('orange_small_churn_data.train')
y = pd.read_csv('orange_small_churn_labels.train', header=None, names=['y'])

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [11]:
data_train = pd.concat([X_train, y_train], axis=1)
data_test = pd.concat([X_test, y_test], axis=1)

In [14]:
data_train.to_csv('data_train.csv', index=False)

In [17]:
data_test.to_csv('hold_out_dataset.csv', index=False)

#### Подумайте и предложите несколько способов (не менее 3х) обработки категориальных признаков, для того, чтобы их можно было использовать при построении модели. Обратите внимание на модуль sklearn.preprocessing. Начать поиски можно с sklearn.preprocessing.OneHotEncoder.

Обработка категориальных признаков зависит от того, какой алгоритм будет выбран. Самым популярным способом является, наверное, one-hot кодирование. Принципом его работы является то, что i-й категориальный признак заменяется на k_i новых бинарных признаков, где k_i - число значений, которое принимает i-й признак. Значением каждого бинарного признака является индикатор события (принимает данный объект это значение или нет). В результате такого кодирования данные становятся разреженными, поэтому стоит использовать те методы, которые способны работать с разреженными данными. Например, логистическая регрессия.

В случае использования случайного леса или решающего дерева есть возможность построения n-арных деревьев. Каждая вершина разбивается на n вершин, где n - число значений, который принимает категориальный признак. Однако из-за того, что вершина разбивается на n дочерних вершин, критерий ошибки будет минимален, если использовать категориальный признак при делении. Поэтому есть другой подход, который разбивает вершину на 2 дочерние. Для этого нужно разбить множество значений на два непересекающихся подмножества. Условием в вершине будет являться принадлежит ли объект левому подмножеству. Однако количество возможных разбиений много (2^n). Чтобы избежать полного перебора, необходимо работать с категоримальными признаками как с вещественными. Для этого значения категориального признака заменяются на натуральные числа, а порог будет разделять множество значений на два подмножества. При этом сортировать значения необходимо по возрастанию доли объектов класса +1.

Существует еще один примитивный способ: произвольно пронуммеровать значения категориального признака. Данный метод часто заканчивается неудачей, так как алгоритмы учитываают не имеющую смысла упорядоченность. Однако такие неустойчивые алгоритмы можно объединить в композиции (например, Bagging).

При использовании метода kNN можно в качестве расстояния между объектами использовать метрику Хемминга.

#### Подумайте, с помощью какой метрики качества лучше всего оценивать качество будущей модели, какой будет ключевая метрика качества? Поясните свой выбор.

Так как мы имеем дело с несбалансированной выборкой, то метрика accuracy не подходит для оценки качества модели.  Также нужно учесть и цену ошибок. В рамках задачи оттока нам важно определить клиентов, которые собираются от нас уйти, чем тех, которые от нас уходить не собираются. Поэтому ошибочное отнесение клиента, который уходить не собирается, к оттоку не так страшно, как если мы пропустим клиента, который собирается от нас уйти. Из этого можно сделать вывод, что метрика качества recall важнее, чем precision. На самом деле в реальных задачах все гораздо сложнее, и игнорировать precision все таки нельзя. На мой взгляд ключевой метрикой качества все таки является площадь под ROC кривой. Во-первых, она устойчива к изменению баланса классов. Во-вторых, она измеряет не оценку класса, а ее принадлежность к классу 1. Так или иначе мы не сможем удержать всех клиентов, и из-за анонимизированности признаков проще будет определить тех клиентов, которые наиболее вероятно собираются от нас уйти. В-третьих, с помощью данной метрики, можно в последствии оценить и precision, и recall, подобрав нужный порог.

#### Какие вспомогательные метрики качества имеет смысл считать, чтобы более полно оценить качество модели, понять, где её слабые стороны и что нужно улучшать?

После того как мы оценили качество с помощью ROC-AUC, можем посчитать и точность, и полноту. И на основании этих данных подобрать необходимые пороги для отнесения клиентов к классу "отток" или "не отток".  Кроме того, очень важным моментом является время обучения. Если две модели имеют примерно одинаковые значения  ROC AUC, то предпочтение отдается той, которая обучилась быстрее.

#### Подберите оптимальную стратегию проведения кросс-валидации: решите, на сколько фолдов вы будете делить выборку? Выберите тип разбиения данных (k-fold, stratified k-fold, shuffle split, leave one out). Поясните ваш выбор.

Разбить выборку можно на 5 фолдов. Это число рекомендуемое. Во-первых, выборка большая, поэтому обучение больше 5 раз займет много времени. Во-вторых, данные на вес золота, поэтому нежелательно терять данные для обучения. Следовательно, при количестве фолдов меньше 5 много данных уйдет на тест, оценки будут сильно смещенными.

Так как для обучения отобрано 32000 объектов, то такой тип разбиения как leave one out исключается однозначно. Выполнять обучение 32000 раз слишком долго и мучительно. Также хочется, чтобы каждый объект попал в обучение, соответственно, shuffle split нежелателен. Выборка у нас несбалансированная, поэтому предпочтение отдается stratified k-fold, чтобы каждый блок имел одинаковое процентное соотношение классов (такое же как и у целой выборки). 